In [1]:
import cv2
import os

def extract_frames(video_path, output_folder, max_frames_per_second):
    video = cv2.VideoCapture(video_path)
    video_name = os.path.splitext(os.path.basename(video_path))[0]  # Extract video name without extension
    if not video.isOpened():
        print(f"Failed to open video: {video_path}")
        return
    fps = int(video.get(cv2.CAP_PROP_FPS))
    if fps == 0:
        print("FPS is zero, cannot proceed with frame extraction.")
        return
    frames_per_second = min(fps, max_frames_per_second)
    frame_interval = max(1, fps // frames_per_second)
    count = 0
    success, image = video.read()
    
    while success:
        if count % frame_interval == 0:
            frame_filename = f"{video_name}_frame{count}.jpg"  # Unique filename for each frame
            cv2.imwrite(os.path.join(output_folder, frame_filename), image)
        success, image = video.read()
        count += 1


In [2]:
def resize_frames(input_folder, output_folder, size=(720, 720)):
    for frame in os.listdir(input_folder):
        path = os.path.join(input_folder, frame)
        img = cv2.imread(path)
        img_resized = cv2.resize(img, size)
        cv2.imwrite(os.path.join(output_folder, frame), img_resized)


In [3]:
real_videos_path = r"D:\DeepFake Detection System\Test_Model\Dataset\Real\Real0"


In [4]:
os.makedirs(r"D:\DeepFake Detection System\Test_Model\Frames\Real", exist_ok=True)
os.makedirs(r"D:\DeepFake Detection System\Test_Model\Frames\Fake", exist_ok=True)
os.makedirs(r"D:\DeepFake Detection System\Test_Model\Frames\Real", exist_ok=True)
os.makedirs(r"D:\DeepFake Detection System\Test_Model\Frames\Fake", exist_ok=True)


In [5]:
def process_videos(video_folder, extracted_folder, resized_folder, frames_per_second, size=(720, 720)):
    # Ensure directories for frames are created
    os.makedirs(extracted_folder, exist_ok=True)
    os.makedirs(resized_folder, exist_ok=True)
    
    for video in os.listdir(video_folder):
        video_path = os.path.join(video_folder, video)
        if video_path.lower().endswith(('.mp4', '.avi')):  # Check for video files
            extract_frames(video_path, extracted_folder, frames_per_second)
            resize_frames(extracted_folder, resized_folder, size)

# Example usage:
process_videos(real_videos_path, r"D:\DeepFake Detection System\Test_Model\Dataset\Real\Real0", r"D:\DeepFake Detection System\Test_Model\Frames\Real", 15)
#process_videos(fake_videos_path, r"D:\DeepFake Detection System\Test_Model\Frames\Fake", r"D:\DeepFake Detection System\Test_Model\Frames\Fake", 15)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [1]:
import cv2
import os
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from cv2 import dnn

In [ ]:

fake_videos_path = r"D:\DeepFake Detection System\Test_Model\Dataset\Fake"

In [5]:
net = dnn.readNetFromCaffe(
    r"D:\DeepFake Detection System\Test_Model\deploy.prototxt",  # Path to the Caffe deploy prototxt file
    r"D:\DeepFake Detection System\Test_Model\res10_300x300_ssd_iter_140000_fp16.caffemodel"  # Path to the Caffe pre-trained model
)

In [7]:
def extract_and_focus_faces(video_path, output_folder, max_frames_per_second, zoom_factor=2):
    video = cv2.VideoCapture(video_path)
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    if not video.isOpened():
        print(f"Failed to open video: {video_path}")
        return
    fps = int(video.get(cv2.CAP_PROP_FPS))
    frames_per_second = min(fps, max_frames_per_second)
    frame_interval = max(1, fps // frames_per_second)
    count = 0
    success, image = video.read()
    
    while success:
        if count % frame_interval == 0:
            h, w = image.shape[:2]
            blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
            net.setInput(blob)
            faces = net.forward()
            for i in range(faces.shape[2]):
                confidence = faces[0, 0, i, 2]
                if confidence > 0.5:
                    box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (x, y, x1, y1) = box.astype("int")
                    
                    # Adjust box dimensions based on zoom factor
                    center_x, center_y = (x + x1) // 2, (y + y1) // 2
                    width = int((x1 - x) * zoom_factor / 2)
                    height = int((y1 - y) * zoom_factor / 2)
                    new_x, new_x1 = max(0, center_x - width), min(w, center_x + width)
                    new_y, new_y1 = max(0, center_y - height), min(h, center_y + height)

                    face = image[new_y:new_y1, new_x:new_x1]
                    resized_face = cv2.resize(face, (720, 720))
                    frame_filename = f"{video_name}_frame{count}_face{i}.jpg"
                    cv2.imwrite(os.path.join(output_folder, frame_filename), resized_face)
        success, image = video.read()
        count += 1

In [9]:
def process_videos_parallel(video_folder, output_folder, frames_per_second):
    os.makedirs(output_folder, exist_ok=True)
    videos = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi'))]
    with ProcessPoolExecutor() as executor:
        executor.map(lambda p: extract_and_focus_faces(*p), [(v, output_folder, frames_per_second) for v in videos])

# Example usage
process_videos_parallel(real_videos_path, r"D:\DeepFake Detection System\Test_Model\Frames\Real", 15)
process_videos_parallel(fake_videos_path, r"D:\DeepFake Detection System\Test_Model\Frames\Fake", 15)

In [2]:
import cv2
import os

def extract_frames_from_video(video_path, output_folder, max_frames_per_second):
    try:
        video = cv2.VideoCapture(video_path)
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        if not video.isOpened():
            print(f"Failed to open video: {video_path}")
            return
        fps = int(video.get(cv2.CAP_PROP_FPS))
        frames_per_second = min(fps, max_frames_per_second)
        frame_interval = max(1, fps // frames_per_second)
        count = 0
        frame_number = 0
        success, image = video.read()
        while success:
            if count % frame_interval == 0:
                frame_filename = f"{video_name}_frame{frame_number}.jpg"
                cv2.imwrite(os.path.join(output_folder, frame_filename), image)
                frame_number += 1
            success, image = video.read()
            count += 1
        video.release()
    except Exception as e:
        print(f"Error processing {video_path}: {e}")

def extract_frames_from_videos(video_folder, output_folder, frames_per_second):
    os.makedirs(output_folder, exist_ok=True)
    videos = [
        os.path.join(video_folder, f)
        for f in os.listdir(video_folder)
        if f.lower().endswith(('.mp4', '.avi'))
    ]
    for v in videos:
        extract_frames_from_video(v, output_folder, frames_per_second)

if __name__ == "__main__":
    real_videos_path = r"D:\DeepFake Detection System\Test_Model\Dataset\Real\Real4"
    fake_videos_path = r"D:\DeepFake Detection System\Test_Model\Dataset\Fake\Fake0"
    real_frames_output = r"D:\DeepFake Detection System\Test_Model\Frames\Real_Frames"
    fake_frames_output = r"D:\DeepFake Detection System\Test_Model\Frames\Fake_Frames"

    # Extract frames from real videos
    #extract_frames_from_videos(real_videos_path, real_frames_output, 15)

    # Extract frames from fake videos
    extract_frames_from_videos(fake_videos_path, fake_frames_output, 15)
